In [1]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import sys
sys.path.append("/home/mylab-pharma/Code/tuele/pan_HDAC/mylab_panHDAC-master/src/common")
from pharmacy_common import PharmacyCommon
#class to encode smiles
common = PharmacyCommon()
from tqdm import tqdm

Import screening data

In [13]:
emolecules_data_path = "../../data/screening_data/emolecules.csv"
emolecules_data = pd.read_csv(emolecules_data_path)
enamine_hits_data_path = "../../data/screening_data/enamine_hits.csv"
enamine_hits_data = pd.read_csv(enamine_hits_data_path)
Pubchem_DB_data_path = "../../data/screening_data/Pubchem_DB.csv"
Pubchem_DB_data = pd.read_csv(Pubchem_DB_data_path)
all_screen_dataset = pd.concat([emolecules_data, enamine_hits_data, Pubchem_DB_data], axis = 0)

In [14]:
len(emolecules_data), len(enamine_hits_data), len(Pubchem_DB_data), len(all_screen_dataset)

(65067, 460160, 59470, 584697)

In [15]:
all_screen_dataset.head(10)

,CID,SMILES
0,33508570,O=C1CCON1
1,29918552,CCC(=O)NO
2,538665,NCC(=O)NO
3,516063,CON(C)C(C)=O
4,480512,ON1C(=O)CCC1=O
5,480512,ON1C(=O)CCC1=O
6,42774990,CC1(C)CONC1=O
7,4864824,CC(C)(C)C(=O)NO
8,32281505,CCC(=O)N(C)OC
9,40180491,CCC(N)C(=O)NO


Checking Duplicate SMILES

In [16]:
smiles_col_name = "SMILES"
cid_col_name = "CID"

In [17]:
screening_smiles = all_screen_dataset["SMILES"].unique()

In [18]:
len(screening_smiles)

542907

Checking if the SMILES are encodedable

In [19]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm

def check_encodable(data) -> np.ndarray:
    result = []
    error_data = []
    with tqdm(total=len(data), desc='Progress') as pbar:
        for i in data:
            try:
                mol = Chem.MolFromSmiles(i)
                fpts = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2,1024))
                result.append(i)
            except:
                error_data.append(i)
            pbar.update(1)  # Update the progress bar
    return result, error_data
encodable_smile, error_smile = check_encodable(screening_smiles)

Progress:   0%|          | 0/542907 [00:00<?, ?it/s]

Progress:   0%|          | 2036/542907 [00:00<03:19, 2716.15it/s][22:23:28] Explicit valence for atom # 14 N, 4, is greater than permitted
[22:23:28] Explicit valence for atom # 13 N, 4, is greater than permitted
[22:23:28] Explicit valence for atom # 15 N, 4, is greater than permitted
Progress:   1%|          | 3710/542907 [00:01<03:28, 2581.32it/s][22:23:29] Explicit valence for atom # 16 N, 4, is greater than permitted
[22:23:29] Explicit valence for atom # 15 N, 4, is greater than permitted
[22:23:29] Explicit valence for atom # 17 N, 4, is greater than permitted
Progress:   1%|          | 5553/542907 [00:02<03:37, 2467.06it/s][22:23:29] Explicit valence for atom # 14 N, 4, is greater than permitted
[22:23:29] Explicit valence for atom # 13 N, 4, is greater than permitted
Progress:   1%|          | 6045/542907 [00:02<03:47, 2357.50it/s][22:23:30] Explicit valence for atom # 11 N, 4, is greater than permitted
[22:23:30] Explicit valence for atom # 19 N, 4, is greater than permitted


In [20]:
print(len(encodable_smile), len(error_smile)) 

542869 38


In [21]:
encoded_df = pd.DataFrame({"SMILES": encodable_smile})
check_encode = common.gen_ecfp4_fpts(encoded_df["SMILES"],1024)
encoded_df.head()


Progress:   0%|          | 0/542869 [00:00<?, ?it/s]

Progress: 100%|██████████| 542869/542869 [03:00<00:00, 3009.78it/s]


,SMILES
0,O=C1CCON1
1,CCC(=O)NO
2,NCC(=O)NO
3,CON(C)C(C)=O
4,ON1C(=O)CCC1=O


In [22]:
error_data = pd.DataFrame({"SMILES":error_smile})
error_data.head(10)

,SMILES
0,CON(C)C(=O)c1cccc(OCC[NH]2CCCC2)c1
1,CON(C)C(=O)c1ccc(OCC[NH]2CCCC2)cc1
2,CON(C)C(=O)c1ccccc1OCC[NH]1CCCC1
3,CON(C)C(=O)c1cccc(C)c1OCC[NH]1CCCC1
4,CON(C)C(=O)C=Cc1cccc(OCC[NH]2CCCC2)c1
5,CON(C)C(=O)C=Cc1ccc(OCC[NH]2CCCC2)cc1
6,CON(C)C(=O)C=Cc1ccccc1OCC[NH]1CCCC1
7,CON(C)C(=O)c1cc(Cl)ccc1OCC[NH]1CCCC1
8,CON(C)C(=O)C=Cc1cc(C)ccc1OCC[NH]1CCCC1
9,CON(C)C(=O)c1ccc(c(OCC[NH]2CCCC2)c1)[N+]([O-])=O


Write to file

In [25]:
len(encoded_df)

542869

In [26]:
# Tính tổng số hàng trong tập dữ liệu
total_rows = len(encoded_df)

# Tính số lượng tập con cần chia
num_splits = (total_rows // 25000) + (1 if total_rows % 25000 != 0 else 0)

# Chia tập dữ liệu thành các tập con
data_splits = []
for i in range(num_splits):
    start = i * 25000
    end = min((i + 1) * 25000, total_rows)
    data_splits.append(encoded_df.iloc[start:end])

# Các tập dữ liệu con được lưu trong danh sách `data_splits`
# Bây giờ bạn có thể sử dụng chúng theo cách mình muốn
for i, split in enumerate(data_splits):
    print(f"Tập dữ liệu con {i+1} có {len(split)} hàng")

Tập dữ liệu con 1 có 25000 hàng
Tập dữ liệu con 2 có 25000 hàng
Tập dữ liệu con 3 có 25000 hàng
Tập dữ liệu con 4 có 25000 hàng
Tập dữ liệu con 5 có 25000 hàng
Tập dữ liệu con 6 có 25000 hàng
Tập dữ liệu con 7 có 25000 hàng
Tập dữ liệu con 8 có 25000 hàng
Tập dữ liệu con 9 có 25000 hàng
Tập dữ liệu con 10 có 25000 hàng
Tập dữ liệu con 11 có 25000 hàng
Tập dữ liệu con 12 có 25000 hàng
Tập dữ liệu con 13 có 25000 hàng
Tập dữ liệu con 14 có 25000 hàng
Tập dữ liệu con 15 có 25000 hàng
Tập dữ liệu con 16 có 25000 hàng
Tập dữ liệu con 17 có 25000 hàng
Tập dữ liệu con 18 có 25000 hàng
Tập dữ liệu con 19 có 25000 hàng
Tập dữ liệu con 20 có 25000 hàng
Tập dữ liệu con 21 có 25000 hàng
Tập dữ liệu con 22 có 17869 hàng


In [27]:
dataset = data_splits[1]
dataset.head(10)

,SMILES
25000,O=C(c1cc(F)ccc1O)N1CCCC(N2CCCS2(=O)=O)C1
25001,COc1ccccc1CCC(=O)Nc1ccc(N2CCCC2)cn1
25002,Cc1cccc(C(=O)Nc2cc(C(N)=O)ccc2N2CCCCC2)n1
25003,O=C(Nc1ccc2nc(CN3CCOCC3)[nH]c2c1)c1cc(Cl)c[nH]1
25004,Cc1ccc(CNC(=O)NCC#Cc2ccccc2)c(C)c1
25005,CN(CC(=O)NC1CCc2cc(F)ccc21)c1ccc(C#N)cn1
25006,COc1ccc(NC(=O)CCN2CCOCC2(C)C)cc1
25007,CCn1nc(C)c(C(=O)N(C)CC(=O)Nc2cc(C)ccn2)c1C
25008,CCc1csc(NC(=O)C2CC2c2cccnc2)n1
25009,Cc1ccc(F)c(NC(=O)C(=O)NCCCN2CCC(O)CC2)c1


In [12]:
with pd.ExcelWriter('../../data/screening_data/emolecules_screen_data.xlsx', engine='openpyxl') as writer:
    all_screen_dataset.to_excel(writer, sheet_name='original_data', index=False)
    encoded_df.to_excel(writer, sheet_name="final_screen_data", index=False)
    error_data.to_excel(writer,sheet_name="unencoded_smiles",index=False)
    